# Размер выборки при тестировании гипотез и способы влияния на него

Для чего вообще управлять размером выборки? Строго говоря, это не совсем корректная формулировка, правильнее было бы сказать "управлять размером минимально достаточной выборки при заданном уровне значимости". Зачем это вообще нужно? Очевидно, что чем меньше наша выборка, тем меньше времени нам нужно на сбор данных, а) тем дешевле будет наш эксперимент б) если он увенчается успехом, тем быстрее мы раскатим результаты в прод, то бишь opportunity cost будет ниже. 

## Доверительные интервалы и как они связаны с размером выборки

Вообще есть несколько интерпретаций понятия "доверительный интервал". самой популярной из которых является следующая - это интервал, в который с вероятностью на выбранном нами уровне значимости попадает значение статистики, для который мы конструируем этот интервал. Проблема в том, что в такой формулировке это не совсем корректное определение. Почему? Вся эта история с интервалом нужна нам только потому, что мы не знаем искомую нами статистику по генеральной совокупности и пытаемся оценить ее, используя выборку. У этой статистики есть вполне определенное значение. Да, мы его не знаем, но тем не менее, разве корректно будет говорить, что (неизвестное нам) значение 8 с вероятностью 90% входит в доверительный интервал \[7,9\]? Очевидно, что это абсурд. Именно поэтому уровень значимости называется confindence level, а не probability, т.к. он не столько про вероятность, сколько про уверенность. 
Я предпочитаю другую интерпретацию, а именно - если мы сделаем множество выборок из нашей генеральной совокупности, то в 99% случаев (при 99% уровне значимости) среднее в популяции попадет в наш доверительный интервал.

Как правило, доверительный интервал записывают в таком формате: $(t-z\sigma^*; t+z\sigma^*)$, где t - это искомая статистика по выборке, $\sigma^*$ - среднеквадратичное отклонение **оцениваемой статистики**, а z - константа, соответствующая выбранному доверительному уровню в исследуемом распределении (те самые значения, которые мы смотрим таблицах распределений). Так, для нормального распределения при уровне значимости 95% $z\approx1.96$.

Стоит остановиться на $\sigma^*$ чуть подробней, потому что это еще один источник постоянной путаницы. Предположим, что у нас есть некоторая выборка со средней $\mu$ и дисперсией $\sigma^2$. Соответственно, мы хотим оценить доверительный интервал для средней по генеральной совокупности $\mu^*$ на уровне значимости 95%. z=1.96, как мы уже знаем, $t=\mu$, осталось понять чему равна $\sigma^*$. Еще раз акцентирую внимание, что мы оцениваем доверительный интервал для **средней** поэтому нас интересует именно распределение средней, а не распределение изначальной величины. Как мы помним из ЦПТ, средняя будет (почти) нормально распределена (собственно, поэтому и z=1.96). Так вот, стандартное отклонение средней - это не то же самое, что стандартное отклонение исходных данных. К счастью, они связаны друг с другом весьма простой формулой (за что тоже нужно благодарить ЦПТ): $\large{\sigma^*=\frac{\sigma}{\sqrt{n}}}$. Таким образом, если $z\sigma^*$ мы обозначим как E, то бишь предельно допустимая ошибка (или минимальный размер эффекта, который мы пытаемся отловить), то путем нехитрых алгебраических образований получаем следующую картину:
\begin{align}
\large{E=\frac{z\sigma}{\sqrt{n}} \implies n=\frac{z^2\sigma^2}{E^2}}
\end{align}

При взгляде на эту формулу становится понятно, что у нас есть три основных способа управлять размером выборки - увеличить предельно допустимую ошибку или снизить уровень значимости (чего делать бы не хотелось), либо уменьшить дисперсию. Упражнения ради накидаем простенький калькулятор размера выборки для заданного датасета.

In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('kc_house_data.csv')
@interact(mu=fixed(df['price'].mean()), conf_level=(0.8, 0.99, 0.01), error_margin=(0.01,1.0,0.005), variance = (df['price'].var()/5, df['price'].var()*2, df['price'].var()/10))
def sample_finder(mu, conf_level: float, error_margin, variance: float):
    #понятно что в нормальном виде эта функция должна принимать просто массив с данными, а не отдельно среднюю и отдельно дисперсию, тут просто хочу показать, как меняется размер выборки с изменением дисперсии
    true_error_margin = mu*error_margin
    z = stats.norm.ppf(0.5+conf_level/2) #это двухвостый уровень значимости, т.к. ppf всегда возвращает однохвостый результат
    return int((z*variance)/(true_error_margin**2))


interactive(children=(FloatSlider(value=0.89, description='conf_level', max=0.99, min=0.8, step=0.01), FloatSl…

С одной стороны как ошибкой, так и размером эффекта, который мы хотим поймать, можно управлять напрямую, с другой стороны, как правило, они продиктованы требованиями бизнеса и нельзя просто так взять и сказать, что мы будет довольствоваться, например, FPR=10%. Именно поэтому основной фокус мы сделаем на управлении дисперсией. Основные (известные мне) методы сокращения дисперсии:

1. Стратификация выборки (как до, так и после теста)
2. CUPED
3. Смена метрики на ту, по которой проще собрать нужное кол-во данных*

 Со звездочокй, т.к. это метод, который не направлен на сокращение дисперсии, он скорее снижает косты по привлечению одного элемента выборки. Например. мы хотим понять, как повлияет новая упаковка на продажи продукта. Вместо этого можем померить кол-во строк чеков с этим продуктом. Очевидно, что в случае строк чеков единиц данных будет больше, чем в случае с дневными или месячными продажами. У Нетфликса был другой пример, когда они оценивали влияние фич на ретеншн. Очевидно, ты не хочешь ждать целый месяц, чтобы понять, уйдет от тебя пользователь или нет, чтобы замерить ретеншн напрямую - это слишком дорого. У них уже было исследование, говорящее о том что ретеншн суперсильно коррелирует с кол-вом часов проведенных за просмотром, и именно эту метрику они использовали как прокси для ретеншена.

## Стратификация выборки

Классическая выборка, как мы помним, получается путем полностью (псевдо)случайного отнесения испытуемых в тестовую и контрольные группы. Очевидно, что, как генеральная совокупность, так и наши выборки могут быть разбиты на какие-то группы (мужчины и женщины, представители разных рас, люди с разным уровнем дохода. пользователи разных браузеров и т.п.). Собственно, разбиение выборки на такого рода группы - это и есть стратификация. Методы бывают разные - иногда мы хотим повторить структуру генеральной совокупности (proportionate sampling), иногда дополнительно скорректировать веса на дисперсию так, чтобы группы с более высокой дисперсией были лучше представлены в нашей выборке (это называется disproportionate sampling). Тогда для proportionate sampling имеем следующее: 

$\large{w_k=\frac{N_k}{N}}$, где $w_k$ - вес группы в выборке, $N_k$ - кол-во членов группы в генеральной совокупности, а N - общее кол-во членов ген. совокупности.

Для наиболее известной разновидности disproportionate sampling, а именно сэмплирования Неймана, имеем:

 $\LARGE{w_k = \frac{N_k\sigma_k}{\sum\limits_{i=1}^{k}{N_k\sigma_k}}}$, то бишь взвешивание идет не просто по кол-ву а по произведению дисперсии и кол-ва.

 Наконец, сэмплирование Неймана является частным случаем т.н. optimum allocation, т.е. такой стратифицированной выборки, которая позволяет добиться максимальной точности при минимальных затратах:

 $\LARGE{w_k = \frac{N_k\sigma_k/\sqrt{c_k}}{\sum\limits_{i=1}^{k}{N_k\sigma_k\sqrt{c_k}}}}$ (корень тоже под знаком суммы).

Теперь, когда мы разобрались с тем, как лучше подбирать веса для страт в выборке, попробуем ответить на вопрос о том, как, собственно, стратификация помогает снизить дисперсию. Мне показалось, что лучше всего будет сделать это с конца: сначала я покажу итоговую формулу и словами объясню, почему это работает, а уже потом приведу математическое доказательство. Итак:

$\large{var_{srs}(Y) = var_{within\,strata}(Y) + var_{between\,strata}(Y)}$,

 т.е. дисперсия при простом случайном сэмплировании разбивается на дисперсию внутри страты и дисперсию между стратами (напоминает ANOVA? Это неспроста). Так вот, при помощи стратификации мы избавляемся от дисперсии между стратами! Не нужно быть гением, чтобы понять, что чем она больше, тем лучше себя показывает стратификация. Теперь перейдем к самому интересному, а именно к (умеренно) строгому математическому доказательству всей этой истории. Для этого, впрочем нам понадобится небольшой экскурс в базовую статистику.

### Закон общей выборки

Итак, закон общей выборки:

$\large{var(Y) = E(var(Y|X)) + var(E(Y|X))}$, где E - это мат. ожидание, а var(Y|X) - условное мат. ожидание. Это очень важный для статистики закон, на котором базируется, например, регрессионный анализ. Чтобы доказать его, нам сначала потребуется доказать два небольших равенства. Первое:

$
\begin{align}
var(X) = E((X-\mu)^2) = E((X-E(X))^2) = E(X^2-2E(X)X+E(X)^2) = E(X^2)-E(2E(X)X)+E(E(X)^2) = E(X^2)-2(E(X)E(X)+E(X)^2) = \boxed{E(X^2) - E(X)^2}
\end{align}
$

Второе:
$
\begin{align}
\underline{E(E(X|Y))} = E(\sum_xxP(X=x|Y)) = \sum_y[\sum_xxP(X=x|Y=y)]*P(Y=y) =  \sum_y\sum_xxP(X=x, Y=y) = \sum_xx\sum_yP(X=x, Y=y) = \sum_xxP(X=x) = \underline{E(X)}
\end{align}
$

Вооружившись этими бесценными знаниями, мы наконец-то можем перейти к доказательству закона общей выборки. Начнем с первого члена:

$\begin{align}
E(var(Y|X)) = E(E(Y^2|X)) - E(E(Y|X)^2) = E(Y^2) - E(Y|X)^2
\end{align}$

Второй:

$\begin{align}
var(E(Y|X)) = E(E(Y|X)^2) - E(E(Y|X))^2 = E(Y|X)^2 - E(Y)^2
\end{align}$

Теперь сложим их:

$\begin{align}
E(var(Y|X)) + var(E(Y|X)) = E(Y^2) - E(Y|X)^2 + E(Y|X)^2 - E(Y)^2 = E(Y^2) - E(Y)^2 = var(Y), ч т.д.
\end{align}$

### Почему стратификация работает

Сначала разберемся с определениями:

- $\mu = E(Y)$ - общее среднее по популяции
- $\mu_k$ - среднее по k-ой страте
- $\sigma^2$ - дисперсия популяции
- $\sigma_k^2$ - дисперсия k-ой страты
- p_k - вес k-ой страты в популяции
- n_k - кол-во членов k-ой популяции
- n - общее кол-во членов популяции. $n=\sum_kn_k$
- $\bar{Y}$ - эстимейт средней по популяции. Обычное среднее арифметическое. $\large{\bar{Y} = \frac{1}{n}\sum_k\sum_jY_{kj}}$
- $\hat{Y}_{strat}$ - эстимейт средней по популяции, но полученный как средневзвешенное. $\large{\hat{Y}_{strat} = \sum_kp_k\bar{Y}_k}$
- srs - simple random sampling, то бишь простая случайная выборка
- strat - стратифицированная выборка

Начнем с того, что покажем, что в случае со стратифицированной выборкой $\large{\hat{Y}_{strat} = \bar{Y}}$:

$
\begin{align}
\large{\hat{Y}_{strat} =\sum_kp_k\bar{Y}_k =  \sum_kp_k\frac{1}{n_k}\sum_jY_{kj} = \sum_k\frac{n_k}{n}\frac{1}{n_k}\sum_jY_{kj} = \frac{1}{n}\sum_k\sum_jY_{kj}=\bar{Y}}
\end{align}
$

Важно понимать, что это верно только для стратифицированной выборки, потому что в случае SRS равенство $\large{p_k=\frac{n_k}{n}}$, очевидно, не выполняется (поскольку выборка случайная, веса страт в выборке не совпадают с таковыми в популяции). Теперь определим дисперсию средней в случае стратифицированной выборки:

$
\begin{align}
\large{var_{strat}(\hat{Y}_{strat}) = \sum_kvar(\hat{Y}_{strat}p_k) = \sum_kp_k^2var(\hat{Y}_{strat}) = \sum_k\frac{n_k^2}{n^2}\frac{\sigma_k^2}{n_k} = \boxed{\frac{1}{n}\sum_kp_k\sigma_k^2}}
\end{align}
$

Воспрользуемся законом общей дисперсии, чтобы разложить дисперсию в случае SRS. Для этого добавим еще парочку обозначений, а именно Z = {1,2 .. K}. Это переменная, которая содержит в себе порядковый номер каждой страты. Понятное дело, что вероятность попадания в страту равна $p_k$, то бишь $P(Z=k) = p_k$. Наконец, введем индикаторную переменную, которая будет равна единице, если Z=k и 0, если это не так. Обозначим ее следующим образом: $I(Z=k)$. Мы наконец готовы к разложению SRS дисперсии.

$
\begin{align}
\large{
    var_{srs}(Y) = E_{srs}(var_{srs}(Y|Z)) + var_{srs}(E_{srs}(Y|Z)) = E_{srs}(\sum_k\sigma_k^2I(z=k)) + var_{srs}(\sum_k\mu_kI(Z=k)) = \sum_k\sigma_k^2E_{srs}(I(Z=k)) + E((\sum_k\mu_kI(Z=k))^2) - E(\sum_k\mu_kI(Z=k))^2 \\ = \sum_k\sigma_k^2p_k + \sum_k\mu_k^2p_k - \sum_k\mu^2 = \sum_k\sigma_k^2p_k + \sum_k\mu_k^2p_k - \mu^2 = \boxed{\sum_k\sigma_k^2p_k + \sum_kp_k(\mu_k-\mu)^2}
}
\end{align}
$

Напоминалка - последнее преобразование это ни что иное как переход от записи дисперсии в виде разницы мат. ожиданий к классической записи с отклонениями от средних. Стоит так же обратить внимание, что пока что мы получили только дисперсию базовую переменной, а не искомую дисперсию средней. К счастью, это делается элементарно:

$
\begin{align}
\large{
var_{srs}(\bar{Y}) = \frac{var_{srs}(Y)}{n} = \frac{1}{n}\sum_k\sigma_k^2p_k + \frac{1}{n}\sum_kp_k(\mu_k-\mu)^2
}
\end{align}
$

Не нужно быть сверхразумом, чтобы заметить, что дисперсия среднего при SRS больше, чем дисперсия при стратифицированной выборке на $\frac{1}{n}\sum_kp_k(\mu_k-\mu)^2$, то бишь на дисперсию между стратами. Кстати, трюк с бинарной переменной принадлежности к страте - это самая, что ни на есть ANOVA.

In [15]:
from sklearn.model_selection import train_test_split
df_trim = df[~df['bedrooms'].isin([33,11,10,9,8])]
df_trim['bedrooms'].groupby(by=df_trim['bedrooms']).count()
x_train, x_test, y_train, y_test = train_test_split(df_trim['price'], df_trim['bedrooms'], test_size=0.02, stratify=df_trim['bedrooms'])
checker = pd.concat([x_test, y_test], axis=1)
checker.groupby(by=checker['bedrooms']).count()

,price
bedrooms,
1,4
2,55
3,197
4,138
5,32
6,5
7,1


### Постстратификация

В реальной жизни, особенно когда идет множество тестов одновременно (что весьма типично для ИТ-компаний), не часто удается стратифицировать выборку на этапе дизайна эксперимента и в таком случае можно прибегнуть к постстратификации. Это работает следующим образом - мы делаем обычное SRS, но потом для расчета средних используем $\large{\hat{Y}_{strat} = \sum_kp_k\bar{Y}_k}$. Т.е. получается, что средние по стратам мы получаем непосредственно из нашего SRS, а вот их веса берем из популяции, таким образом давая больший приоритет недопредставленным стратам и меньший тем стратам, членов которых в выборке оказалось больше, чем в популяции. Выведем дисперсию для этого случая:

$
\begin{align}
\large{
var_{srs}(\hat{Y}_{strat}) = E(var_{srs}(\hat{Y}_{strat}|n_1,...,n_K)) + var_{srs}(E(\hat{Y}_{strat}|n_1,...,n_K)) = E(\sum_kp_k^2var_{srs}(\bar{Y_k}|n_k)) + var_{srs}(\sum_kp_k\mu_k) = E(\sum_kp_k^2\frac{\sigma^2_k}{n_k}) + var_{srs}(\mu) = \boxed{\sum_kp_k^2\sigma_k^2E(\frac{1}{n_k})}
}
\end{align}
$

$p_k$ и $\sigma_k^2$ у нас есть. Получается, нужно оценить только $\large{E(\frac{1}{n_k})}$. Для этого отметим обратим внимание на то, что $n_k$ - это случайная величина Бернулли с мат. ожиданием $np_k$. Само собой, сам факт нахождения $n_k$ в знаменателе, говорит нам о том, что наша выборка обязана содержать хотя бы одного члена каждой страты. Для мат. стороны вопроса это просто еще одно ограничение, но на практике за этим реально важно следить (как вариант, совсем маленькие страты можно схлопывать в одну - старая-добрая категория "Others"). Вооружившись этим знанием, разложим $\large{E(\frac{1}{n_k})}$ в ряд Тейлора около точки $np_k$:

$
\begin{align}
\large{
    E(\frac{1}{n_k}) = E(\frac{1}{np_k} - \frac{1}{n^2p_k^2}(n_k-np_k) + \frac{1}{n^3p_k^3}(n_k-np_k)^2 + o(\frac{1}{n^2}))
}
\end{align}
$

Обратим внимание, что мат. ожидание $(n_k-np_k)$ равно нулю по определению (собственно, в т.ч. поэтому мы для получения дисперсии возводим это выражение в квадрат). В свою очередь, $E(n_k-np_k)^2$ - это дисперсия. Наконец $o(\frac{1}{n^2}))$ - это [small o в нотации Ландау](https://brilliant.org/wiki/big-o-notation/), что, грубо говоря, позволяет нам откинуть этот член, практически не потеряв в точности аппроксимации. Таким образом, получаем следующую картину:

$
\begin{align}
\large{
    E(\frac{1}{n_k}) = E(\frac{1}{np_k}) + \frac{1}{n^3p_k^3}np_k(1-p_k) = \frac{1}{n_k} + \frac{1}{n^2p_k^2}(1-p_k)
}
\end{align}
$

Наконец, для дисперсии в случае постстратификации получаем:

$
\begin{align}
\large{
    \sum_kp_k^2\sigma_k^2E(\frac{1}{n_k}) = \sum_k\frac{p_k^2\sigma_k^2}{n_k} + \frac{p_k^2\sigma_k^2}{n^2p_k^2}(1-p_k) = \boxed{(\frac{1}{n}\sum_kp_k\sigma_k^2) + (\frac{1}{n^2}\sum_k\sigma_k^2(1-p_k))}
}
\end{align}
$

Получается, что мы всегда можем найти такое n, для которого $\large{\frac{1}{n^2}\sum_k\sigma_k^2(1-p_k)\leq\frac{1}{n}\sum_kp_k(\mu_k-\mu)^2}$. Поскольку n - это размер популяции, то, как правило, он у нас всегда дотсаточно большой для того, чтобы то неравенство выполнялось. И пропишу еще раз - страта - это не просто какой-то способ разделить наши данные на группы, а что-то связанное с исследуемой метрикой. Например если E(ARPU) сильно разное в разных городах, есть смысл использовать город как страту, если же такой связи нет, то и вся эта история со стратификацией нам не поможет (например, Бабушкин говорит, что в X5 стратификация уменьшила им дисперсию всего лишь на 1%). Итак, в конечном итоге, мы имеем следующее неравенство:

$
\begin{align}
\large{
    \boxed{
        var_{strat}\leq var_{poststrat}\leq var_{srs}
    }
}
\end{align}
$

При этом для достаточно больших n, разница между стратификацией и постстратификацией гораздо менее существенна, чем разница между SRS и постстратификацией. Теперь можно перейти к методу, который, по сути, стал стандартом индустрии и частным случаем которого стратификация и является.

## CUPED